In [1]:
import os
import numpy as np
import efficientnet.tfkeras
from tensorflow.keras.models import load_model
from tensorflow.keras import models
import tensorflow as tf
from focal_loss import BinaryFocalLoss

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"]=""

In [3]:
model_dir ='/media/tohn/SSD/ModelTrainByImages/R2_4/models/B5R2b5_AN_1FC_fold4_3.h5'
model = load_model(model_dir)
height = width = model.input_shape[1]

x = model.get_layer('head_pooling').output
prediction_layer = model.output
model2 = models.Model(inputs= model.input, outputs=[prediction_layer,x]) 

In [4]:
#ไม่ต้องเปลี่ยนก็ได้ เพราะเอา Label มาใช้เฉยๆ 
# validation
import pandas as pd
base_dir  = '/media/tohn/SSD/Images/Image2/validation/'
dataframe = pd.read_csv( '/home/yupaporn/codes/USAI/Validationdf_fold1_3.csv')
validation_dir = os.path.join(base_dir, 'validation')

#Train
train_df = pd.read_csv( '/home/yupaporn/codes/USAI/Traindf_fold1_3.csv')
base_dir0 = '/media/tohn/SSD/Images/Image1/'
os.chdir(base_dir0)
train_dir = os.path.join(base_dir0, 'train')

In [5]:
import pandas as pd
df0 = pd.read_csv (r'/home/yupaporn/codes/USAI/Validationdf_fold4_3.csv')
print(df0 .shape)
dataframe = df0[(df0['Path Crop']!='None' )&(df0['Path Crop']!='Nan')]
# a = df01[df01['Case'].between(1, 10)]
# dataframe = df01[df01['Abs Position']!='P8']
print(dataframe.shape)
print('Normal: ',dataframe[dataframe['Class']=='Normal'].shape)
print('Abnormal: ',dataframe[dataframe['Class']=='Abnormal'].shape)
dataframe.head(5)

(1056, 22)
(1056, 22)
Normal:  (572, 22)
Abnormal:  (484, 22)


,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Case,Abs Position,Sub Position,Class,Sub_class,Path Full,Path Crop,...,tagName,originalImage,left,top,width,height,Rleft,Rtop,Rwidth,Rheight
0,0,300,300,3,P1,P1,Abnormal,AB01,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,...,MildFattyLiver,https://irisprodseatraining.blob.core.windows....,220.0,117.0,706.0,475.0,0.218085,0.130945,0.698582,0.527888
1,1,302,302,3,P1,P1,Abnormal,AB02,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,...,ModerateFattyLiver,https://irisprodseatraining.blob.core.windows....,194.0,101.0,541.0,630.0,0.198147,0.113258,0.549816,0.701010
2,2,310,310,3,P1,P1,Abnormal,AB03,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,...,SevereFattyLiver,https://irisprodseatraining.blob.core.windows....,213.0,131.0,533.0,547.0,0.216199,0.145581,0.541436,0.608081
3,3,313,313,3,P1,P1,Abnormal,AB04,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,...,Cirrhosis,https://irisprodseatraining.blob.core.windows....,176.0,156.0,632.0,625.0,0.178381,0.173864,0.637363,0.694949
4,4,316,316,3,P1,P1,Abnormal,AB06,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,...,PDF2,https://irisprodseatraining.blob.core.windows....,249.0,85.0,475.0,437.0,0.249811,0.095076,0.476364,0.486869


In [6]:
# dataframe.info()

In [6]:
batch_size = 64
epochs = 10

from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=30,
      width_shift_range=0.2,
      height_shift_range=0.2,
      brightness_range=[0.5,1.5],
      shear_range=0.4,
      zoom_range=0.2,
      horizontal_flip=False,
      fill_mode='nearest')

train_generator = train_datagen.flow_from_dataframe(
        dataframe = dataframe,
        directory = train_dir,
        x_col = 'Path Crop',
        y_col = 'Class',
        target_size = (height, width),
        batch_size=batch_size,
        color_mode= 'rgb',
        class_mode='categorical')

#label
labels = (train_generator.class_indices)
labels = dict((v,k.replace("C","")) for k,v in labels.items())
print(labels)

Found 1056 validated image filenames belonging to 2 classes.
{0: 'Abnormal', 1: 'Normal'}


In [7]:
from tensorflow.keras.preprocessing import image
def predict_image(img_path):
    # Read the image and resize it
    img = image.load_img(img_path, target_size=(height, width))
    # Convert it to a Numpy array with target shape.
    x = image.img_to_array(img)
    # Reshape
    x = x.reshape((1,) + x.shape)
    x /= 255.
    result = model2.predict([x])
    result1 = result[0][0]
    result2 = result[1][0]

    return result1, result2

#Predict
pred_list = list()
prob_list = list()
vectorlist = list()
img_path=dataframe['Path Crop'].tolist()
for i in range(0,len(img_path)):
    result1,result2 = predict_image(img_path[i])
    result = np.argmax(result1)
    pred_list.append(labels[result])
    prob_list.append(result1[result])
    vectorlist.append(result2.tolist())
    
dataframe['category'] = pred_list
dataframe['Prob'] = prob_list

In [8]:
vector_array = np.array(vectorlist)
vector_array

array([[ 0.01672857, -0.17232947, -0.11835328, ...,  0.06245726,
        -0.11820588, -0.0168822 ],
       [-0.10786887, -0.12088429,  0.02993098, ..., -0.12530668,
        -0.03858192, -0.01144609],
       [-0.1460783 , -0.15053776,  0.11208129, ..., -0.07037959,
        -0.0980221 , -0.16199569],
       ...,
       [ 0.4323965 ,  0.45547089, -0.21788432, ...,  0.22409232,
        -0.24253635,  0.21953717],
       [ 0.40908536,  0.36992607,  0.02527384, ...,  0.17636168,
        -0.05457454,  0.47235695],
       [ 0.11847439,  0.77360499, -0.06973302, ...,  0.95542604,
         0.03519964,  0.69118172]])

In [9]:
dataset = pd.DataFrame(vector_array[:, :])
dataset

,0,1,2,3,4,5,6,7,8,9,...,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047
0,0.016729,-0.172329,-0.118353,-0.135590,-0.146592,-0.217166,-0.076235,-0.014151,-0.196719,-0.155609,...,-0.085173,-0.203584,-0.182687,-0.016323,-0.116521,-0.209810,0.033223,0.062457,-0.118206,-0.016882
1,-0.107869,-0.120884,0.029931,0.028059,-0.209720,-0.184374,0.082525,-0.193829,-0.201076,-0.007949,...,-0.145172,-0.056531,0.003197,-0.146088,-0.205128,-0.172549,0.042853,-0.125307,-0.038582,-0.011446
2,-0.146078,-0.150538,0.112081,0.228713,-0.200509,-0.195468,-0.036482,-0.216962,-0.223975,0.104877,...,-0.194535,-0.126278,-0.056003,-0.160584,-0.224714,-0.142666,-0.185281,-0.070380,-0.098022,-0.161996
3,-0.113831,-0.146489,0.004781,-0.049187,-0.195438,-0.158852,0.384925,-0.201790,-0.192398,-0.030004,...,-0.145096,-0.027276,-0.149160,-0.121812,-0.193086,0.259221,-0.118337,-0.115629,-0.039028,-0.077751
4,0.121595,0.051315,-0.153521,0.000540,-0.093299,-0.078620,0.147926,0.167608,0.017269,-0.137484,...,0.125532,-0.056640,-0.174644,-0.019680,-0.015877,0.112325,-0.013576,0.004979,0.024083,0.040686
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1051,0.408185,0.515072,-0.072548,-0.077460,0.205401,0.430854,-0.135483,0.046386,0.457932,-0.172103,...,0.516265,-0.167018,-0.014770,0.118542,0.521898,-0.181926,-0.118138,0.274917,-0.137121,0.418283
1052,0.638564,0.725674,-0.144380,-0.105379,0.412248,0.371223,-0.109901,0.664233,0.592711,-0.151829,...,0.469494,-0.174162,-0.122292,0.044018,0.255639,-0.114318,0.015215,0.348346,-0.010909,0.868089
1053,0.432397,0.455471,-0.217884,-0.184195,0.174189,-0.009831,-0.106081,-0.063430,0.108088,-0.209061,...,0.178937,-0.197461,-0.131204,0.123369,0.271046,-0.197102,-0.211313,0.224092,-0.242536,0.219537
1054,0.409085,0.369926,0.025274,-0.085282,0.465753,0.488475,-0.178869,0.176259,0.706152,-0.175701,...,0.309840,-0.203842,-0.109584,0.185172,0.511559,-0.181666,-0.197932,0.176362,-0.054575,0.472357


In [10]:
a = dataframe['Class'].tolist()
b = dataframe['Sub Position'].tolist()
c = dataframe['Views'].tolist()
d = dataframe['Sub_class'].tolist()
dataset['Class'] = a 
dataset['SubPosition'] = b
dataset['Views'] = c
dataset['Sub_class'] = d

In [11]:
dataset

,0,1,2,3,4,5,6,7,8,9,...,2042,2043,2044,2045,2046,2047,Class,SubPosition,Views,Sub_class
0,0.016729,-0.172329,-0.118353,-0.135590,-0.146592,-0.217166,-0.076235,-0.014151,-0.196719,-0.155609,...,-0.116521,-0.209810,0.033223,0.062457,-0.118206,-0.016882,Abnormal,P1,FP-A,AB01
1,-0.107869,-0.120884,0.029931,0.028059,-0.209720,-0.184374,0.082525,-0.193829,-0.201076,-0.007949,...,-0.205128,-0.172549,0.042853,-0.125307,-0.038582,-0.011446,Abnormal,P1,FP-A,AB02
2,-0.146078,-0.150538,0.112081,0.228713,-0.200509,-0.195468,-0.036482,-0.216962,-0.223975,0.104877,...,-0.224714,-0.142666,-0.185281,-0.070380,-0.098022,-0.161996,Abnormal,P1,FP-A,AB03
3,-0.113831,-0.146489,0.004781,-0.049187,-0.195438,-0.158852,0.384925,-0.201790,-0.192398,-0.030004,...,-0.193086,0.259221,-0.118337,-0.115629,-0.039028,-0.077751,Abnormal,P1,FP-A,AB04
4,0.121595,0.051315,-0.153521,0.000540,-0.093299,-0.078620,0.147926,0.167608,0.017269,-0.137484,...,-0.015877,0.112325,-0.013576,0.004979,0.024083,0.040686,Abnormal,P1,FP-A,AB06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1051,0.408185,0.515072,-0.072548,-0.077460,0.205401,0.430854,-0.135483,0.046386,0.457932,-0.172103,...,0.521898,-0.181926,-0.118138,0.274917,-0.137121,0.418283,Normal,P8,FP-E,Normal
1052,0.638564,0.725674,-0.144380,-0.105379,0.412248,0.371223,-0.109901,0.664233,0.592711,-0.151829,...,0.255639,-0.114318,0.015215,0.348346,-0.010909,0.868089,Normal,P8,FP-E,Normal
1053,0.432397,0.455471,-0.217884,-0.184195,0.174189,-0.009831,-0.106081,-0.063430,0.108088,-0.209061,...,0.271046,-0.197102,-0.211313,0.224092,-0.242536,0.219537,Normal,P8,FP-E,Normal
1054,0.409085,0.369926,0.025274,-0.085282,0.465753,0.488475,-0.178869,0.176259,0.706152,-0.175701,...,0.511559,-0.181666,-0.197932,0.176362,-0.054575,0.472357,Normal,P8,FP-E,Normal


In [12]:
os.chdir( '/home/yupaporn/codes/USAI/')
testdf = dataset
testdf.to_csv('FVvalidation_AN_RF_AN_fold4_3.csv')